In [1]:
import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.3.1"
os.environ["PATH"] = r"C:\Program Files\R\R-4.3.1\bin\x64" + ";" + os.environ["PATH"]

In [2]:
import pandas as pd
import random
from tqdm import tqdm
import wandb

from rpy2.robjects.packages import STAP
from rpy2.robjects import pandas2ri

import sklearn

import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
pandas2ri.activate()

### Reading and preprocessing spreadsheets

In [4]:
df_female_train = pd.read_csv('./FemaleTrainR.csv')
df_female_val = pd.read_csv('./FemaleValR.csv')
df_female_test = pd.read_csv('./FemaleTestR.csv')

In [5]:
df_male_train = pd.read_csv('./MaleTrainR.csv')
df_male_val = pd.read_csv('./MaleValR.csv')
df_male_test = pd.read_csv('./MaleTestR.csv')

In [6]:
df_train_R = pd.concat([df_female_train, df_male_train],
                       ignore_index=True)
df_val_R = pd.concat([df_female_val, df_male_val],
                     ignore_index=True)
df_test_R = pd.concat([df_female_test, df_male_test],
                      ignore_index=True)

#### Example of the train dataframe containing female and male audios 

In [12]:
df_train_R

,sound.files,selec,start,end,gender
0,audio_10831_1.wav,0,0,3,1
1,audio_10831_2.wav,0,0,3,1
2,audio_10831_3.wav,0,0,3,1
3,audio_10832_1.wav,0,0,3,1
4,audio_10832_2.wav,0,0,3,1
...,...,...,...,...,...
4675,audio_301_1.wav,0,0,3,0
4676,audio_3020_1.wav,0,0,3,0
4677,audio_3021_1.wav,0,0,3,0
4678,audio_3021_2.wav,0,0,3,0


In [17]:
# 
# df_val_R_shuffled = sklearn.utils.shuffle(df_val_R, random_state=99)
# 
# df_test_R_shuffled = sklearn.utils.shuffle(df_test_R, random_state=99)

### Converting dataframes into right type for rpy2

In [7]:
trainR = pandas2ri.py2rpy(df_train_R)
valR = pandas2ri.py2rpy(df_val_R)
testR = pandas2ri.py2rpy(df_test_R)

### Running R script

In [8]:
from specan3 import funcR

specan3 = STAP(funcR, "specan3")

specan3_train = specan3.specan3(trainR)
specan3_val = specan3.specan3(valR)
specan3_test = specan3.specan3(testR)

 easuring acoustic parameters: |                                                  | 0 % ~calculating   |+                                                 | 1 % ~15m 02s       |+                                                 | 2 % ~13m 55s       |++                                                | 3 % ~13m 21s       |++                                                | 4 % ~12m 58s       |+++                                               | 5 % ~12m 44s       |+++                                               | 6 % ~12m 26s       |++++                                              | 7 % ~12m 08s       |++++                                              | 8 % ~11m 52s       |+++++                                             | 9 % ~11m 32s       |+++++                                             | 10% ~11m 20s       |++++++                                            | 11% ~11m 13s       |++++++                                            | 12% ~11m 04s       |+++++++                         

#### Example of the output numpy array from the script

In [9]:
specan3_train

sound.files,selec,duration,...,maxdom,dfrange,modindx
...,...,...,...,...,...,...


### Converting numpy arrays into pandas dataframes

In [10]:
res_trainR = pandas2ri.rpy2py(specan3_train)
res_trainR

,sound.files,selec,duration,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
1,audio_10831_1.wav,0,3.0,0.133557,0.078894,0.147509,0.055066,0.189225,0.134160,9.830947,...,0.133557,0.0,0.092083,0.015748,0.250000,0.078804,0.000000,0.476562,0.476562,0.131148
2,audio_10831_2.wav,0,3.0,0.113482,0.080591,0.110465,0.048057,0.165197,0.117139,4.235740,...,0.113482,0.0,0.110163,0.015764,0.192771,0.075068,0.000000,0.507812,0.507812,0.166434
3,audio_10831_3.wav,0,3.0,0.113590,0.085415,0.107128,0.032372,0.186555,0.154184,2.069623,...,0.113590,0.0,0.104470,0.025357,0.271186,0.066051,0.000000,0.125000,0.125000,0.270833
4,audio_10832_1.wav,0,3.0,0.127018,0.087797,0.120810,0.040715,0.212920,0.172205,1.667893,...,0.127018,0.0,0.141555,0.015873,0.250000,0.148777,0.000000,0.593750,0.593750,0.275718
5,audio_10832_2.wav,0,3.0,0.162497,0.073489,0.196567,0.109464,0.205578,0.096114,3.921250,...,0.162497,0.0,0.191397,0.022161,0.271186,0.726562,0.007812,6.164062,6.156250,0.198835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676,audio_301_1.wav,0,3.0,0.150582,0.065094,0.148844,0.109464,0.213254,0.103790,3.628258,...,0.150582,0.0,0.118524,0.026846,0.271186,0.365412,0.000000,4.351562,4.351562,0.125246
4677,audio_3020_1.wav,0,3.0,0.212256,0.052163,0.234279,0.191895,0.248629,0.056734,2.088487,...,0.212256,0.0,0.122631,0.028623,0.266667,1.039062,0.242188,6.687500,6.445312,0.220985
4678,audio_3021_1.wav,0,3.0,0.207966,0.055345,0.226937,0.152181,0.251967,0.099785,1.545044,...,0.207966,0.0,0.129050,0.043478,0.266667,1.277832,0.000000,6.367188,6.367188,0.290710
4679,audio_3021_2.wav,0,3.0,0.213915,0.056425,0.228939,0.190894,0.261311,0.070417,1.562229,...,0.213915,0.0,0.092447,0.017391,0.207792,1.691236,0.195312,6.218750,6.023438,0.247907


### Processing train output dataframe

In [11]:
res_train = res_trainR.drop(['selec', 'duration'], axis=1)
res_train

,sound.files,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,...,centroid,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
1,audio_10831_1.wav,0.133557,0.078894,0.147509,0.055066,0.189225,0.134160,9.830947,159.585815,0.953479,...,0.133557,0.0,0.092083,0.015748,0.250000,0.078804,0.000000,0.476562,0.476562,0.131148
2,audio_10831_2.wav,0.113482,0.080591,0.110465,0.048057,0.165197,0.117139,4.235740,40.727151,0.951087,...,0.113482,0.0,0.110163,0.015764,0.192771,0.075068,0.000000,0.507812,0.507812,0.166434
3,audio_10831_3.wav,0.113590,0.085415,0.107128,0.032372,0.186555,0.154184,2.069623,10.892234,0.964277,...,0.113590,0.0,0.104470,0.025357,0.271186,0.066051,0.000000,0.125000,0.125000,0.270833
4,audio_10832_1.wav,0.127018,0.087797,0.120810,0.040715,0.212920,0.172205,1.667893,9.222255,0.959946,...,0.127018,0.0,0.141555,0.015873,0.250000,0.148777,0.000000,0.593750,0.593750,0.275718
5,audio_10832_2.wav,0.162497,0.073489,0.196567,0.109464,0.205578,0.096114,3.921250,20.333699,0.902344,...,0.162497,0.0,0.191397,0.022161,0.271186,0.726562,0.007812,6.164062,6.156250,0.198835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676,audio_301_1.wav,0.150582,0.065094,0.148844,0.109464,0.213254,0.103790,3.628258,24.968572,0.924027,...,0.150582,0.0,0.118524,0.026846,0.271186,0.365412,0.000000,4.351562,4.351562,0.125246
4677,audio_3020_1.wav,0.212256,0.052163,0.234279,0.191895,0.248629,0.056734,2.088487,7.076406,0.884992,...,0.212256,0.0,0.122631,0.028623,0.266667,1.039062,0.242188,6.687500,6.445312,0.220985
4678,audio_3021_1.wav,0.207966,0.055345,0.226937,0.152181,0.251967,0.099785,1.545044,5.004890,0.899583,...,0.207966,0.0,0.129050,0.043478,0.266667,1.277832,0.000000,6.367188,6.367188,0.290710
4679,audio_3021_2.wav,0.213915,0.056425,0.228939,0.190894,0.261311,0.070417,1.562229,4.870377,0.897199,...,0.213915,0.0,0.092447,0.017391,0.207792,1.691236,0.195312,6.218750,6.023438,0.247907


In [13]:
df_train_merged = res_train.merge(df_train_R, how = 'left', 
                                  left_on='sound.files', 
                                  right_on='sound.files')
df_train_merged

,sound.files,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,...,maxfun,meandom,mindom,maxdom,dfrange,modindx,selec,start,end,gender
0,audio_10831_1.wav,0.133557,0.078894,0.147509,0.055066,0.189225,0.134160,9.830947,159.585815,0.953479,...,0.250000,0.078804,0.000000,0.476562,0.476562,0.131148,0,0,3,1
1,audio_10831_2.wav,0.113482,0.080591,0.110465,0.048057,0.165197,0.117139,4.235740,40.727151,0.951087,...,0.192771,0.075068,0.000000,0.507812,0.507812,0.166434,0,0,3,1
2,audio_10831_3.wav,0.113590,0.085415,0.107128,0.032372,0.186555,0.154184,2.069623,10.892234,0.964277,...,0.271186,0.066051,0.000000,0.125000,0.125000,0.270833,0,0,3,1
3,audio_10832_1.wav,0.127018,0.087797,0.120810,0.040715,0.212920,0.172205,1.667893,9.222255,0.959946,...,0.250000,0.148777,0.000000,0.593750,0.593750,0.275718,0,0,3,1
4,audio_10832_2.wav,0.162497,0.073489,0.196567,0.109464,0.205578,0.096114,3.921250,20.333699,0.902344,...,0.271186,0.726562,0.007812,6.164062,6.156250,0.198835,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,audio_301_1.wav,0.150582,0.065094,0.148844,0.109464,0.213254,0.103790,3.628258,24.968572,0.924027,...,0.271186,0.365412,0.000000,4.351562,4.351562,0.125246,0,0,3,0
4676,audio_3020_1.wav,0.212256,0.052163,0.234279,0.191895,0.248629,0.056734,2.088487,7.076406,0.884992,...,0.266667,1.039062,0.242188,6.687500,6.445312,0.220985,0,0,3,0
4677,audio_3021_1.wav,0.207966,0.055345,0.226937,0.152181,0.251967,0.099785,1.545044,5.004890,0.899583,...,0.266667,1.277832,0.000000,6.367188,6.367188,0.290710,0,0,3,0
4678,audio_3021_2.wav,0.213915,0.056425,0.228939,0.190894,0.261311,0.070417,1.562229,4.870377,0.897199,...,0.207792,1.691236,0.195312,6.218750,6.023438,0.247907,0,0,3,0


In [14]:
df_train_merged.columns

Index(['sound.files', 'meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew',
       'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'peakf', 'meanfun',
       'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx',
       'selec', 'start', 'end', 'gender'],
      dtype='object')

In [16]:
df_train_merged = df_train_merged.drop(['sound.files', 'selec', 
                                        'start', 'end'], axis=1)
df_train_merged

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
0,0.133557,0.078894,0.147509,0.055066,0.189225,0.134160,9.830947,159.585815,0.953479,0.756729,...,0.0,0.092083,0.015748,0.250000,0.078804,0.000000,0.476562,0.476562,0.131148,1
1,0.113482,0.080591,0.110465,0.048057,0.165197,0.117139,4.235740,40.727151,0.951087,0.694607,...,0.0,0.110163,0.015764,0.192771,0.075068,0.000000,0.507812,0.507812,0.166434,1
2,0.113590,0.085415,0.107128,0.032372,0.186555,0.154184,2.069623,10.892234,0.964277,0.775908,...,0.0,0.104470,0.025357,0.271186,0.066051,0.000000,0.125000,0.125000,0.270833,1
3,0.127018,0.087797,0.120810,0.040715,0.212920,0.172205,1.667893,9.222255,0.959946,0.735926,...,0.0,0.141555,0.015873,0.250000,0.148777,0.000000,0.593750,0.593750,0.275718,1
4,0.162497,0.073489,0.196567,0.109464,0.205578,0.096114,3.921250,20.333699,0.902344,0.536839,...,0.0,0.191397,0.022161,0.271186,0.726562,0.007812,6.164062,6.156250,0.198835,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,0.150582,0.065094,0.148844,0.109464,0.213254,0.103790,3.628258,24.968572,0.924027,0.574567,...,0.0,0.118524,0.026846,0.271186,0.365412,0.000000,4.351562,4.351562,0.125246,0
4676,0.212256,0.052163,0.234279,0.191895,0.248629,0.056734,2.088487,7.076406,0.884992,0.314636,...,0.0,0.122631,0.028623,0.266667,1.039062,0.242188,6.687500,6.445312,0.220985,0
4677,0.207966,0.055345,0.226937,0.152181,0.251967,0.099785,1.545044,5.004890,0.899583,0.327399,...,0.0,0.129050,0.043478,0.266667,1.277832,0.000000,6.367188,6.367188,0.290710,0
4678,0.213915,0.056425,0.228939,0.190894,0.261311,0.070417,1.562229,4.870377,0.897199,0.332301,...,0.0,0.092447,0.017391,0.207792,1.691236,0.195312,6.218750,6.023438,0.247907,0


In [19]:
df_train = sklearn.utils.shuffle(df_train_merged, random_state=99)
df_train

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
4025,0.178633,0.058182,0.189559,0.120477,0.229607,0.109130,2.111708,8.980167,0.913346,0.418257,...,0.0,0.100808,0.020699,0.242424,0.570724,0.101562,6.812500,6.710938,0.136076,0
993,0.198949,0.041437,0.199905,0.172872,0.233945,0.061073,1.622246,5.294816,0.883193,0.291510,...,0.0,0.164817,0.015952,0.231884,1.330882,0.171875,5.851562,5.679688,0.360752,1
403,0.210127,0.043004,0.216925,0.181549,0.235614,0.054064,1.786626,6.417706,0.889656,0.278893,...,0.0,0.182341,0.017937,0.275862,0.300164,0.000000,0.664062,0.664062,0.316263,1
2443,0.174596,0.056812,0.180215,0.127819,0.220262,0.092443,2.158536,8.641442,0.916238,0.485560,...,0.0,0.129149,0.028829,0.225352,0.348090,0.101562,1.820312,1.718750,0.155152,0
2095,0.190948,0.056744,0.182217,0.145507,0.245626,0.100119,1.190036,4.181933,0.920620,0.419908,...,0.0,0.139372,0.039120,0.219178,0.343290,0.125000,0.656250,0.531250,0.287684,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,0.221265,0.027453,0.222598,0.205578,0.240620,0.035042,1.823762,5.526717,0.837741,0.114920,...,0.0,0.184659,0.016895,0.275862,1.214583,0.187500,5.750000,5.562500,0.318921,1
3457,0.148515,0.059986,0.127485,0.116472,0.191228,0.074756,3.784406,19.026308,0.871798,0.443737,...,0.0,0.110944,0.015795,0.210526,0.645312,0.085938,6.960938,6.875000,0.172378,0
1768,0.218440,0.040091,0.223933,0.204243,0.248629,0.044386,1.955926,6.859694,0.862636,0.243767,...,0.0,0.200594,0.048780,0.271186,0.466712,0.203125,1.320312,1.117188,0.232994,1
1737,0.218364,0.046873,0.227604,0.207247,0.249631,0.042384,2.152666,7.686288,0.864819,0.297163,...,0.0,0.211521,0.071111,0.275862,0.437500,0.203125,1.304688,1.101562,0.224711,1


### Processing validation output dataframe

In [20]:
res_valR = pandas2ri.rpy2py(specan3_val)
res_valR

,sound.files,selec,duration,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
1,audio_21525_1.wav,0,3.0,0.224695,0.047868,0.230274,0.206913,0.258308,0.051395,1.694384,...,0.224695,0.0,0.182119,0.051118,0.271186,0.359375,0.195312,0.664062,0.468750,0.272222
2,audio_21526_1.wav,0,3.0,0.218447,0.058324,0.231943,0.214589,0.251299,0.036710,2.203279,...,0.218447,0.0,0.194605,0.027119,0.250000,0.806362,0.179688,5.976562,5.796875,0.097761
3,audio_21527_1.wav,0,3.0,0.232033,0.044446,0.240286,0.204577,0.265983,0.061406,2.300159,...,0.232033,0.0,0.216338,0.112676,0.275862,0.380078,0.195312,0.703125,0.507812,0.264777
4,audio_21528_1.wav,0,3.0,0.232303,0.046820,0.247294,0.211585,0.264315,0.052729,2.141446,...,0.232303,0.0,0.202153,0.135593,0.271186,0.371324,0.242188,0.554688,0.312500,0.355000
5,audio_21529_1.wav,0,3.0,0.228318,0.055432,0.244958,0.207580,0.267986,0.060405,3.280261,...,0.228318,0.0,0.182917,0.016393,0.271186,0.477941,0.218750,1.460938,1.242188,0.259434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,audio_5510_1.wav,0,3.0,0.178471,0.063492,0.197235,0.117139,0.230608,0.113468,1.010811,...,0.178471,0.0,0.118920,0.029851,0.275862,1.234375,0.093750,4.617188,4.523438,0.289983
1557,audio_5511_1.wav,0,3.0,0.175401,0.057290,0.155852,0.142169,0.224601,0.082431,2.904013,...,0.175401,0.0,0.125155,0.038462,0.186047,0.429688,0.125000,2.906250,2.781250,0.177297
1558,audio_5512_1.wav,0,3.0,0.180324,0.060594,0.175209,0.128153,0.236949,0.108796,1.118284,...,0.180324,0.0,0.121888,0.029304,0.271186,0.683594,0.101562,4.421875,4.320312,0.249229
1559,audio_5513_1.wav,0,3.0,0.175605,0.056232,0.174207,0.131490,0.223600,0.092110,0.858922,...,0.175605,0.0,0.133606,0.019975,0.238806,0.827257,0.117188,5.125000,5.007812,0.254565


In [21]:
res_val = res_valR.drop(['selec', 'duration'], axis=1)
res_val

,sound.files,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,...,centroid,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
1,audio_21525_1.wav,0.224695,0.047868,0.230274,0.206913,0.258308,0.051395,1.694384,5.056327,0.870315,...,0.224695,0.0,0.182119,0.051118,0.271186,0.359375,0.195312,0.664062,0.468750,0.272222
2,audio_21526_1.wav,0.218447,0.058324,0.231943,0.214589,0.251299,0.036710,2.203279,8.211853,0.877827,...,0.218447,0.0,0.194605,0.027119,0.250000,0.806362,0.179688,5.976562,5.796875,0.097761
3,audio_21527_1.wav,0.232033,0.044446,0.240286,0.204577,0.265983,0.061406,2.300159,9.141529,0.852427,...,0.232033,0.0,0.216338,0.112676,0.275862,0.380078,0.195312,0.703125,0.507812,0.264777
4,audio_21528_1.wav,0.232303,0.046820,0.247294,0.211585,0.264315,0.052729,2.141446,7.454452,0.865963,...,0.232303,0.0,0.202153,0.135593,0.271186,0.371324,0.242188,0.554688,0.312500,0.355000
5,audio_21529_1.wav,0.228318,0.055432,0.244958,0.207580,0.267986,0.060405,3.280261,16.519999,0.869023,...,0.228318,0.0,0.182917,0.016393,0.271186,0.477941,0.218750,1.460938,1.242188,0.259434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,audio_5510_1.wav,0.178471,0.063492,0.197235,0.117139,0.230608,0.113468,1.010811,3.291075,0.937175,...,0.178471,0.0,0.118920,0.029851,0.275862,1.234375,0.093750,4.617188,4.523438,0.289983
1557,audio_5511_1.wav,0.175401,0.057290,0.155852,0.142169,0.224601,0.082431,2.904013,12.811037,0.880570,...,0.175401,0.0,0.125155,0.038462,0.186047,0.429688,0.125000,2.906250,2.781250,0.177297
1558,audio_5512_1.wav,0.180324,0.060594,0.175209,0.128153,0.236949,0.108796,1.118284,4.422185,0.931975,...,0.180324,0.0,0.121888,0.029304,0.271186,0.683594,0.101562,4.421875,4.320312,0.249229
1559,audio_5513_1.wav,0.175605,0.056232,0.174207,0.131490,0.223600,0.092110,0.858922,3.317293,0.939273,...,0.175605,0.0,0.133606,0.019975,0.238806,0.827257,0.117188,5.125000,5.007812,0.254565


In [23]:
df_val_merged = res_val.merge(df_val_R, how = 'left', 
                              left_on='sound.files', 
                              right_on='sound.files')
df_val_merged

,sound.files,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,...,maxfun,meandom,mindom,maxdom,dfrange,modindx,selec,start,end,gender
0,audio_21525_1.wav,0.224695,0.047868,0.230274,0.206913,0.258308,0.051395,1.694384,5.056327,0.870315,...,0.271186,0.359375,0.195312,0.664062,0.468750,0.272222,0,0,3,1
1,audio_21526_1.wav,0.218447,0.058324,0.231943,0.214589,0.251299,0.036710,2.203279,8.211853,0.877827,...,0.250000,0.806362,0.179688,5.976562,5.796875,0.097761,0,0,3,1
2,audio_21527_1.wav,0.232033,0.044446,0.240286,0.204577,0.265983,0.061406,2.300159,9.141529,0.852427,...,0.275862,0.380078,0.195312,0.703125,0.507812,0.264777,0,0,3,1
3,audio_21528_1.wav,0.232303,0.046820,0.247294,0.211585,0.264315,0.052729,2.141446,7.454452,0.865963,...,0.271186,0.371324,0.242188,0.554688,0.312500,0.355000,0,0,3,1
4,audio_21529_1.wav,0.228318,0.055432,0.244958,0.207580,0.267986,0.060405,3.280261,16.519999,0.869023,...,0.271186,0.477941,0.218750,1.460938,1.242188,0.259434,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,audio_5510_1.wav,0.178471,0.063492,0.197235,0.117139,0.230608,0.113468,1.010811,3.291075,0.937175,...,0.275862,1.234375,0.093750,4.617188,4.523438,0.289983,0,0,3,0
1556,audio_5511_1.wav,0.175401,0.057290,0.155852,0.142169,0.224601,0.082431,2.904013,12.811037,0.880570,...,0.186047,0.429688,0.125000,2.906250,2.781250,0.177297,0,0,3,0
1557,audio_5512_1.wav,0.180324,0.060594,0.175209,0.128153,0.236949,0.108796,1.118284,4.422185,0.931975,...,0.271186,0.683594,0.101562,4.421875,4.320312,0.249229,0,0,3,0
1558,audio_5513_1.wav,0.175605,0.056232,0.174207,0.131490,0.223600,0.092110,0.858922,3.317293,0.939273,...,0.238806,0.827257,0.117188,5.125000,5.007812,0.254565,0,0,3,0


In [25]:
df_val_merged = df_val_merged.drop(['sound.files', 'selec', 
                                    'start', 'end'], axis=1)
df_val_merged

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
0,0.224695,0.047868,0.230274,0.206913,0.258308,0.051395,1.694384,5.056327,0.870315,0.291056,...,0.0,0.182119,0.051118,0.271186,0.359375,0.195312,0.664062,0.468750,0.272222,1
1,0.218447,0.058324,0.231943,0.214589,0.251299,0.036710,2.203279,8.211853,0.877827,0.359837,...,0.0,0.194605,0.027119,0.250000,0.806362,0.179688,5.976562,5.796875,0.097761,1
2,0.232033,0.044446,0.240286,0.204577,0.265983,0.061406,2.300159,9.141529,0.852427,0.221396,...,0.0,0.216338,0.112676,0.275862,0.380078,0.195312,0.703125,0.507812,0.264777,1
3,0.232303,0.046820,0.247294,0.211585,0.264315,0.052729,2.141446,7.454452,0.865963,0.265874,...,0.0,0.202153,0.135593,0.271186,0.371324,0.242188,0.554688,0.312500,0.355000,1
4,0.228318,0.055432,0.244958,0.207580,0.267986,0.060405,3.280261,16.519999,0.869023,0.378858,...,0.0,0.182917,0.016393,0.271186,0.477941,0.218750,1.460938,1.242188,0.259434,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,0.178471,0.063492,0.197235,0.117139,0.230608,0.113468,1.010811,3.291075,0.937175,0.532558,...,0.0,0.118920,0.029851,0.275862,1.234375,0.093750,4.617188,4.523438,0.289983,0
1556,0.175401,0.057290,0.155852,0.142169,0.224601,0.082431,2.904013,12.811037,0.880570,0.376863,...,0.0,0.125155,0.038462,0.186047,0.429688,0.125000,2.906250,2.781250,0.177297,0
1557,0.180324,0.060594,0.175209,0.128153,0.236949,0.108796,1.118284,4.422185,0.931975,0.483893,...,0.0,0.121888,0.029304,0.271186,0.683594,0.101562,4.421875,4.320312,0.249229,0
1558,0.175605,0.056232,0.174207,0.131490,0.223600,0.092110,0.858922,3.317293,0.939273,0.525665,...,0.0,0.133606,0.019975,0.238806,0.827257,0.117188,5.125000,5.007812,0.254565,0


In [26]:
df_val = sklearn.utils.shuffle(df_val_merged, random_state=99)
df_val

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
520,0.219175,0.043650,0.228605,0.189225,0.254970,0.065745,1.770236,6.873874,0.882019,0.256931,...,0.0,0.212481,0.045845,0.271186,0.805099,0.171875,6.187500,6.015625,0.154545,1
81,0.243427,0.039529,0.252634,0.237282,0.265983,0.028701,2.803833,11.395809,0.817192,0.188912,...,0.0,0.199526,0.082474,0.275862,0.680990,0.195312,4.945312,4.750000,0.148800,1
360,0.211070,0.054491,0.232610,0.193897,0.245626,0.051728,2.448970,9.728006,0.890481,0.400684,...,0.0,0.227120,0.100629,0.275862,0.896662,0.164062,5.289062,5.125000,0.179297,1
498,0.207730,0.046308,0.210250,0.187223,0.233945,0.046722,1.722116,5.620900,0.892216,0.348063,...,0.0,0.196458,0.036036,0.271186,0.849095,0.164062,5.390625,5.226562,0.202873,1
787,0.203255,0.054083,0.207247,0.187557,0.242622,0.055066,2.584237,11.717237,0.897910,0.374686,...,0.0,0.104005,0.018244,0.253968,1.123641,0.195312,4.937500,4.742188,0.272802,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,0.198963,0.051921,0.217592,0.147843,0.236281,0.088439,1.999578,7.189500,0.882766,0.282113,...,0.0,0.124977,0.018801,0.250000,1.125391,0.132812,6.875000,6.742188,0.191451,0
1192,0.139613,0.082320,0.163528,0.082765,0.187557,0.104791,2.774699,13.256798,0.911132,0.523718,...,0.0,0.098635,0.020539,0.271186,0.328497,0.000000,0.554688,0.554688,0.256338,0
1209,0.155493,0.078076,0.178212,0.092777,0.195900,0.103123,2.598401,10.317746,0.914667,0.548983,...,0.0,0.105099,0.017448,0.258065,0.304688,0.000000,0.562500,0.562500,0.388889,0
1059,0.178390,0.062481,0.168868,0.126818,0.233611,0.106794,1.307550,4.331740,0.918466,0.452920,...,0.0,0.126047,0.039409,0.266667,0.214844,0.000000,0.468750,0.468750,0.187037,0


### Processing test output dataframe

In [27]:
res_testR = pandas2ri.rpy2py(specan3_test)
res_testR

,sound.files,selec,duration,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
1,audio_5351_1.wav,0,3.0,0.199718,0.047559,0.190226,0.172872,0.239952,0.067080,1.921239,...,0.199718,0.0,0.144761,0.015671,0.262295,0.606120,0.171875,2.664062,2.492188,0.158450
2,audio_5352_1.wav,0,3.0,0.222640,0.046298,0.232277,0.194565,0.262312,0.067747,1.140921,...,0.222640,0.0,0.196969,0.101266,0.275862,0.895833,0.000000,6.851562,6.851562,0.194568
3,audio_5353_1.wav,0,3.0,0.239156,0.045808,0.256639,0.220930,0.270656,0.049726,3.605247,...,0.239156,0.0,0.195947,0.031008,0.275862,0.934896,0.195312,6.898438,6.703125,0.127137
4,audio_5354_1.wav,0,3.0,0.188237,0.035217,0.193230,0.181549,0.198570,0.017020,4.390578,...,0.188237,0.0,0.189664,0.032990,0.266667,0.957237,0.164062,6.953125,6.789062,0.223245
5,audio_5356_1.wav,0,3.0,0.239849,0.036013,0.249297,0.225268,0.264315,0.039046,2.280044,...,0.239849,0.0,0.230668,0.113475,0.275862,0.313879,0.195312,0.507812,0.312500,0.295312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,audio_8448_1.wav,0,3.0,0.174845,0.054471,0.159523,0.146841,0.207580,0.060739,2.966620,...,0.174845,0.0,0.117964,0.018957,0.163265,0.242839,0.101562,0.492188,0.390625,0.287273
1557,audio_8449_1.wav,0,3.0,0.183899,0.061197,0.166198,0.146841,0.244958,0.098117,3.852444,...,0.183899,0.0,0.106846,0.041026,0.152381,0.224219,0.085938,0.421875,0.335938,0.255814
1558,audio_8450_1.wav,0,3.0,0.174109,0.060044,0.192563,0.106794,0.214589,0.107795,2.359603,...,0.174109,0.0,0.089152,0.031621,0.130081,0.207031,0.000000,0.421875,0.421875,0.380471
1559,audio_8_1.wav,0,3.0,0.218243,0.045689,0.234946,0.187557,0.251633,0.064076,2.250663,...,0.218243,0.0,0.140005,0.061776,0.246154,0.388021,0.226562,1.695312,1.468750,0.197627


In [28]:
res_test = res_testR.drop(['selec', 'duration'], axis=1)
res_test

,sound.files,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,...,centroid,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
1,audio_5351_1.wav,0.199718,0.047559,0.190226,0.172872,0.239952,0.067080,1.921239,7.126641,0.895461,...,0.199718,0.0,0.144761,0.015671,0.262295,0.606120,0.171875,2.664062,2.492188,0.158450
2,audio_5352_1.wav,0.222640,0.046298,0.232277,0.194565,0.262312,0.067747,1.140921,3.236791,0.888545,...,0.222640,0.0,0.196969,0.101266,0.275862,0.895833,0.000000,6.851562,6.851562,0.194568
3,audio_5353_1.wav,0.239156,0.045808,0.256639,0.220930,0.270656,0.049726,3.605247,18.173039,0.836531,...,0.239156,0.0,0.195947,0.031008,0.275862,0.934896,0.195312,6.898438,6.703125,0.127137
4,audio_5354_1.wav,0.188237,0.035217,0.193230,0.181549,0.198570,0.017020,4.390578,25.433239,0.798928,...,0.188237,0.0,0.189664,0.032990,0.266667,0.957237,0.164062,6.953125,6.789062,0.223245
5,audio_5356_1.wav,0.239849,0.036013,0.249297,0.225268,0.264315,0.039046,2.280044,8.135181,0.830428,...,0.239849,0.0,0.230668,0.113475,0.275862,0.313879,0.195312,0.507812,0.312500,0.295312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,audio_8448_1.wav,0.174845,0.054471,0.159523,0.146841,0.207580,0.060739,2.966620,14.267492,0.900638,...,0.174845,0.0,0.117964,0.018957,0.163265,0.242839,0.101562,0.492188,0.390625,0.287273
1557,audio_8449_1.wav,0.183899,0.061197,0.166198,0.146841,0.244958,0.098117,3.852444,23.386638,0.918454,...,0.183899,0.0,0.106846,0.041026,0.152381,0.224219,0.085938,0.421875,0.335938,0.255814
1558,audio_8450_1.wav,0.174109,0.060044,0.192563,0.106794,0.214589,0.107795,2.359603,9.740310,0.908177,...,0.174109,0.0,0.089152,0.031621,0.130081,0.207031,0.000000,0.421875,0.421875,0.380471
1559,audio_8_1.wav,0.218243,0.045689,0.234946,0.187557,0.251633,0.064076,2.250663,8.923878,0.885236,...,0.218243,0.0,0.140005,0.061776,0.246154,0.388021,0.226562,1.695312,1.468750,0.197627


In [29]:
df_test_merged = res_test.merge(df_test_R, how = 'left', 
                                left_on='sound.files', 
                                right_on='sound.files')
df_test_merged

,sound.files,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,...,maxfun,meandom,mindom,maxdom,dfrange,modindx,selec,start,end,gender
0,audio_5351_1.wav,0.199718,0.047559,0.190226,0.172872,0.239952,0.067080,1.921239,7.126641,0.895461,...,0.262295,0.606120,0.171875,2.664062,2.492188,0.158450,0,0,3,1
1,audio_5352_1.wav,0.222640,0.046298,0.232277,0.194565,0.262312,0.067747,1.140921,3.236791,0.888545,...,0.275862,0.895833,0.000000,6.851562,6.851562,0.194568,0,0,3,1
2,audio_5353_1.wav,0.239156,0.045808,0.256639,0.220930,0.270656,0.049726,3.605247,18.173039,0.836531,...,0.275862,0.934896,0.195312,6.898438,6.703125,0.127137,0,0,3,1
3,audio_5354_1.wav,0.188237,0.035217,0.193230,0.181549,0.198570,0.017020,4.390578,25.433239,0.798928,...,0.266667,0.957237,0.164062,6.953125,6.789062,0.223245,0,0,3,1
4,audio_5356_1.wav,0.239849,0.036013,0.249297,0.225268,0.264315,0.039046,2.280044,8.135181,0.830428,...,0.275862,0.313879,0.195312,0.507812,0.312500,0.295312,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,audio_8448_1.wav,0.174845,0.054471,0.159523,0.146841,0.207580,0.060739,2.966620,14.267492,0.900638,...,0.163265,0.242839,0.101562,0.492188,0.390625,0.287273,0,0,3,0
1556,audio_8449_1.wav,0.183899,0.061197,0.166198,0.146841,0.244958,0.098117,3.852444,23.386638,0.918454,...,0.152381,0.224219,0.085938,0.421875,0.335938,0.255814,0,0,3,0
1557,audio_8450_1.wav,0.174109,0.060044,0.192563,0.106794,0.214589,0.107795,2.359603,9.740310,0.908177,...,0.130081,0.207031,0.000000,0.421875,0.421875,0.380471,0,0,3,0
1558,audio_8_1.wav,0.218243,0.045689,0.234946,0.187557,0.251633,0.064076,2.250663,8.923878,0.885236,...,0.246154,0.388021,0.226562,1.695312,1.468750,0.197627,0,0,3,0


In [30]:
df_test_merged = df_test_merged.drop(['sound.files', 'selec', 
                                      'start', 'end'], axis=1)
df_test_merged

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
0,0.199718,0.047559,0.190226,0.172872,0.239952,0.067080,1.921239,7.126641,0.895461,0.358011,...,0.0,0.144761,0.015671,0.262295,0.606120,0.171875,2.664062,2.492188,0.158450,1
1,0.222640,0.046298,0.232277,0.194565,0.262312,0.067747,1.140921,3.236791,0.888545,0.276710,...,0.0,0.196969,0.101266,0.275862,0.895833,0.000000,6.851562,6.851562,0.194568,1
2,0.239156,0.045808,0.256639,0.220930,0.270656,0.049726,3.605247,18.173039,0.836531,0.247375,...,0.0,0.195947,0.031008,0.275862,0.934896,0.195312,6.898438,6.703125,0.127137,1
3,0.188237,0.035217,0.193230,0.181549,0.198570,0.017020,4.390578,25.433239,0.798928,0.242510,...,0.0,0.189664,0.032990,0.266667,0.957237,0.164062,6.953125,6.789062,0.223245,1
4,0.239849,0.036013,0.249297,0.225268,0.264315,0.039046,2.280044,8.135181,0.830428,0.169399,...,0.0,0.230668,0.113475,0.275862,0.313879,0.195312,0.507812,0.312500,0.295312,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,0.174845,0.054471,0.159523,0.146841,0.207580,0.060739,2.966620,14.267492,0.900638,0.407231,...,0.0,0.117964,0.018957,0.163265,0.242839,0.101562,0.492188,0.390625,0.287273,0
1556,0.183899,0.061197,0.166198,0.146841,0.244958,0.098117,3.852444,23.386638,0.918454,0.450815,...,0.0,0.106846,0.041026,0.152381,0.224219,0.085938,0.421875,0.335938,0.255814,0
1557,0.174109,0.060044,0.192563,0.106794,0.214589,0.107795,2.359603,9.740310,0.908177,0.442799,...,0.0,0.089152,0.031621,0.130081,0.207031,0.000000,0.421875,0.421875,0.380471,0
1558,0.218243,0.045689,0.234946,0.187557,0.251633,0.064076,2.250663,8.923878,0.885236,0.243835,...,0.0,0.140005,0.061776,0.246154,0.388021,0.226562,1.695312,1.468750,0.197627,0


In [31]:
df_test = sklearn.utils.shuffle(df_test_merged, random_state=99)
df_test

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,peakf,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,gender
520,0.244094,0.034427,0.249631,0.229607,0.268319,0.038713,2.316825,8.636644,0.822272,0.150208,...,0.0,0.232994,0.050000,0.275862,0.259983,0.210938,0.351562,0.140625,0.261438,1
81,0.226827,0.038115,0.224267,0.208582,0.254970,0.046389,1.875130,6.111939,0.844976,0.198688,...,0.0,0.198184,0.024316,0.275862,0.331342,0.203125,0.757812,0.554688,0.196244,1
360,0.233559,0.036462,0.235948,0.215256,0.259309,0.044052,1.933781,5.922207,0.835692,0.174098,...,0.0,0.199022,0.017260,0.266667,0.532609,0.203125,6.648438,6.445312,0.099504,1
498,0.232581,0.040548,0.243290,0.209249,0.263313,0.054064,2.144191,8.301671,0.857130,0.216789,...,0.0,0.233023,0.146789,0.271186,0.344210,0.179688,0.929688,0.750000,0.319444,1
787,0.175955,0.059368,0.163528,0.123814,0.228605,0.104791,1.375895,4.860583,0.928281,0.485266,...,0.0,0.106928,0.021192,0.238806,0.684211,0.085938,5.351562,5.265625,0.130651,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,0.184209,0.062385,0.195232,0.137163,0.236949,0.099785,1.013867,3.661669,0.947272,0.596413,...,0.0,0.123920,0.044568,0.228571,0.620888,0.156250,4.734375,4.578125,0.140406,0
1192,0.177205,0.053989,0.196234,0.141836,0.208248,0.066412,3.700158,19.644676,0.902335,0.450338,...,0.0,0.094633,0.016949,0.275862,0.199405,0.132812,0.382812,0.250000,0.164474,0
1209,0.108150,0.077272,0.054398,0.050060,0.150179,0.100119,19.191943,404.739562,0.712307,0.380846,...,0.0,0.117123,0.015779,0.219178,0.046875,0.046875,0.046875,0.000000,0.000000,0
1059,0.171035,0.065421,0.180548,0.115137,0.229273,0.114136,1.004786,3.275404,0.950842,0.645316,...,0.0,0.112259,0.046377,0.213333,0.581414,0.093750,4.187500,4.093750,0.109627,0
